# Introdução ao Pandas - Viagens do Governo | Tratamento de dados

*Esse notebook usa o arquivo sobre [viagens de funcionários do governo](http://www.portaltransparencia.gov.br/viagens) disponibilizado no portal da transparência.*

In [2]:
import pandas as pd

In [3]:
df_viagem = pd.read_csv('viagens_2019.csv', encoding='latin-1', sep=';')

**Saber informações sobre uma coluna específica**

In [4]:
df_viagem['Valor passagens']

0         3406,33
1         2925,83
2         2760,02
3         2875,92
4         2420,48
           ...   
719256    2313,42
719257    1600,42
719258    3663,62
719259    1179,57
719260    2529,42
Name: Valor passagens, Length: 719261, dtype: object

**Mudar as virgulas dos valores por pontos**

In [8]:
# o str é pra sinalizar que a troca do replace é uma string
df_viagem['Valor passagens'] = df_viagem['Valor passagens'].str.replace(',','.')

**Mudar o tipo de dado da coluna**

In [10]:
df_viagem['Valor passagens'] = df_viagem['Valor passagens'].astype(float)

**Contar quantas vezes aparece uma determinada coisa**

In [ ]:
df_viagem['Destinos'].value_counts()

**Separar as cidades dos Estados ou Países**<br><br>
O `split()` guarda o resultado em uma **lista**

In [18]:
# O parametro 'expand = True' indica que os elementos devem ser separados em diferentes colunas
col = df_viagem['Destinos'].str.split('/',1,expand = True)

**Incluir a coluna nova no dataframe**

In [16]:
df_viagem['cidade_destino'] = col[0]

**Separar conteudos diferentes que estão na mesma coluna**

A coluna do dataframe possui valores nulos, estados e países misturados. Para resolver isso é preciso identificar **padrões**. No caso desse dataframe, o padrão é: <br>
- **Estados** tem só **2 caracteres**
- **Paises** estão escritos com nome completo, ou seja tem mais caracteres
- Valores **nulos** estão escritos **None**

Antes de tudo, vou armazenar o conteudo bagunçado em uma coluna que chamarei de 'Provisoria'

In [21]:
df_viagem['Provisoria'] = col[1]

**Criando um dataframe para cada conteudo separado**<br><br>
Todos os dataframes vão ter uma coluna chamda 'Provisoria' com o conteudo separado por conteudo que foi filtrado a partir do dataframe original 'df_viagem'. Para isso filtrarei usando a função `len()`. Essa função apresenta a quantidade de conteúdo que for passado por parametro.

Criando um dataframe com o conteudo de estados 

In [24]:
#Estou colocando na coluna estado o conteudo da coluna provisória que tenha apenas dois caracteres
estado = df_viagem[df_viagem['Provisoria'].str.len()==2]

Agora vou criar um dataframe com o conteudo de paises, incluindo nele todo conteudo que tenha informações maior que dois caracteres

In [27]:
pais = df_viagem[df_viagem['Provisoria'].str.len() > 2]

E por fim vou criar um dataframe com o conteudo None. A função `isnull()` identifica os valores nulos no dataframe. No caso do exemplo está identificando na coluna 'Provisoria' 

In [28]:
nulo = df_viagem[df_viagem['Provisoria'].isnull()]

**Renomeado as colunas dos dataframes**

Se na coluna 'Provisoria' do dataframe 'estado' só tem o conteudo relacionado aos estados, basta renomear a coluna 'Provisoria' para 'estado'.

In [32]:
#No dicionario é passado na chave o nome da coluna antiga e no valor o nome que quer renomear
#O inplace serve para tornar a mudança definitiva

estado.rename(columns = {'Provisoria' : 'estado'}, inplace = True)

Todas os estados são relacionados ao Brasil. Então vou criar dentro do dataframe 'estado' a coluna 'pais' e incluir o conteudo 'Brasil'

In [ ]:
estado['país'] = 'Brasil'

No dataframe 'pais' vou renomear a coluna 'Provisoria' para 'país'

In [ ]:
pais.rename(columns = {'Provisoria' : 'país'}, inplace = True)

Como os outros paises não tem a informação do estado (no caso desse dataframe), vou criar uma coluna 'estado' e incluir o conteudo 'Sem informação'

In [ ]:
pais['estado'] = 'Sem informação'

No dataframe nulo, vou renomear a coluna 'Provisoria' para 'estado' 

In [ ]:
nulo.rename(columns = {'Provisoria' : 'estado'}, inplace = True)

E inserir uma coluna 'país' com o conteudo 'Sem informação'

In [ ]:
nulo['país'] = 'Sem informação'

Vou mudar o conteudo 'None' da coluna 'Estado' para 'Sem informação'

In [ ]:
nulo['estado'] = 'Sem informação'

**Ordenando Colunas** <br><br>
A ordem das colunas novas nos novos dataframes deveria ser 'cidade_destino', 'estado' e 'pais', mas essa ordem está diferente no dataframe 'pais'. Vou reordenar para todos ficarem iguais. 

1. Pegarei o nome de todas as colunas

In [ ]:
pais.columns

2. Agora copio todas as colunas dentro dos colchetes duplos na ordem que eu considerar certa

In [43]:
pais = pais[['Identificador do processo de viagem', 'Situação',
       'Código do órgão superior', 'Nome do órgão superior',
       'Código órgão solicitante', 'Nome órgão solicitante', 'CPF viajante',
       'Nome', 'Cargo', 'Período - Data de início', 'Período - Data de fim',
       'Destinos', 'Motivo', 'Valor diárias', 'Valor passagens',
       'Valor outros gastos', 'cidade_destino', 'estado', 'país']]

**Juntar os 3 dataframes em 1**

In [51]:
#ignore_true serve pra ignorar o indice de cada dataframe, se não vai ser impossivel junta-los

df_final = pd.concat([estado, pais, nulo], ignore_index = True)

**Removendo uma coluna**<br>

Agora não faz mais sentido ter a coluna 'Destinos' afinal ela já foi tratada. Usarei o método `drop()` para fazer isso.

In [55]:
#O axis indica se eu quero fazer minha alteração em uma linha ou em uma coluna. 1 é coluna, 0 é linha.

df_final.drop('Destinos', inplace = True, axis = 1)

**Salvando um arquivo em csv**<br>

É importante depois de tratado gerar um arquivo novo para não precisar rodar tudo e novo.

In [57]:
df_final.to_csv('viagens_tratado_2019.csv', encoding = 'latin-1', sep = ';')